Основа со случайным выбором банкоматов

In [24]:
import random
import numpy as np
import heapq

# **1 Класс ATM и создание банкоматов**

In [17]:
class ATM:
    def __init__(self, atm_id, receive_bin_capacity, dispense_bin_capacity,
                 receive_mean, receive_variance,
                 dispense_mean, dispense_variance):
        self.atm_id = atm_id
        self.receive_bin_capacity = receive_bin_capacity
        self.dispense_bin_capacity = dispense_bin_capacity
        self.receive_bin_level = receive_bin_capacity / 2  # Начальное заполнение на половину
        self.dispense_bin_level = dispense_bin_capacity / 2
        self.receive_mean = receive_mean
        self.receive_variance = receive_variance
        self.dispense_mean = dispense_mean
        self.dispense_variance = dispense_variance

    def days_until_service_needed(self):
        if self.receive_mean > 0:
            days_to_overflow = (self.receive_bin_capacity - self.receive_bin_level) / self.receive_mean
        else:
            days_to_overflow = float('inf')
        if self.dispense_mean > 0:
            days_to_empty = self.dispense_bin_level / self.dispense_mean
        else:
            days_to_empty = float('inf')
        days_until_service = min(days_to_overflow, days_to_empty)
        return days_until_service

'''  atm_id: Уникальный идентификатор банкомата.
receive_bin_capacity и dispense_bin_capacity: Вместимость бункеров для приема и выдачи.
receive_bin_level и dispense_bin_level: Начальные уровни заполнения бункеров (в нашем случае — 50% от емкости).
receive_mean, receive_variance: Среднее значение и дисперсия для поступлений наличности (количество банкнот).
dispense_mean, dispense_variance: Среднее значение и дисперсия для выдачи наличности (количество банкнот).'''


'  atm_id: Уникальный идентификатор банкомата.\nreceive_bin_capacity и dispense_bin_capacity: Вместимость бункеров для приема и выдачи.\nreceive_bin_level и dispense_bin_level: Начальные уровни заполнения бункеров (в нашем случае — 50% от емкости).\nreceive_mean, receive_variance: Среднее значение и дисперсия для поступлений наличности (количество банкнот).\ndispense_mean, dispense_variance: Среднее значение и дисперсия для выдачи наличности (количество банкнот).'

# Генерация банкоматов

In [18]:
num_atms = 1000
atms = []

for atm_id in range(num_atms):
    receive_bin_capacity = random.randint(1000, 5000)
    dispense_bin_capacity = random.randint(1000, 5000)
    receive_mean = random.uniform(100, 500)
    receive_variance = random.uniform(10, 50)
    dispense_mean = random.uniform(100, 500)
    dispense_variance = random.uniform(10, 50)
    atm = ATM(
        atm_id=atm_id,
        receive_bin_capacity=receive_bin_capacity,
        dispense_bin_capacity=dispense_bin_capacity,
        receive_mean=receive_mean,
        receive_variance=receive_variance,
        dispense_mean=dispense_mean,
        dispense_variance=dispense_variance
    )
    atm.x = random.uniform(0, 10)  # Координаты в пределах 10 км x 10 км
    atm.y = random.uniform(0, 10)
    atms.append(atm)

# **2. Прогноз обслуживания банкоматов на несколько дней**

In [19]:
NUM_DAYS = 5
SERVICE_TIME_PER_ATM = 15  # минут
WORKING_DAY_DURATION = 8 * 60  # минут
NUM_GROUPS = 5
AVERAGE_SPEED = 30  # км/ч

atms_needing_service = {day: [] for day in range(NUM_DAYS)}

In [20]:
for day in range(NUM_DAYS):
    for atm in atms:
        if day == 0:
            days_until_service = atm.days_until_service_needed()
            if days_until_service <= 1:
                atms_needing_service[day].append(atm)
        else:
            atm.receive_bin_level += atm.receive_mean
            atm.dispense_bin_level -= atm.dispense_mean
            atm.receive_bin_level = min(atm.receive_bin_level, atm.receive_bin_capacity)
            atm.dispense_bin_level = max(atm.dispense_bin_level, 0)
            if atm.receive_bin_level >= atm.receive_bin_capacity or atm.dispense_bin_level <= 0:
                atms_needing_service[day].append(atm)
                atm.receive_bin_level = atm.receive_bin_capacity / 2
                atm.dispense_bin_level = atm.dispense_bin_capacity / 2

'''Первый день (day = 0):
Для каждого банкомата вызываем метод days_until_service_needed(). Если требуется обслуживание в течение дня, добавляем банкомат в список обслуживания на сегодняшний день.

Последующие дни (day > 0):
Обновляем уровни бункеров, добавляя банкноты в бункер для приема (receive_bin_level += receive_mean) и изымая из бункера для выдачи (dispense_bin_level -= dispense_mean).
Проверяем, не переполнился ли бункер для приема и не опустел ли бункер для выдачи.
Если да, банкомат добавляется в список обслуживания на этот день, и уровни бункеров сбрасываются до начальных значений (наполовину заполненные).
'''

'Первый день (day = 0): \nДля каждого банкомата вызываем метод days_until_service_needed(). Если требуется обслуживание в течение дня, добавляем банкомат в список обслуживания на сегодняшний день.\n\nПоследующие дни (day > 0):\nОбновляем уровни бункеров, добавляя банкноты в бункер для приема (receive_bin_level += receive_mean) и изымая из бункера для выдачи (dispense_bin_level -= dispense_mean).\nПроверяем, не переполнился ли бункер для приема и не опустел ли бункер для выдачи. \nЕсли да, банкомат добавляется в список обслуживания на этот день, и уровни бункеров сбрасываются до начальных значений (наполовину заполненные).\n'

# **3. Построение маршрутов для инкассаторов**

In [21]:
for day in range(NUM_DAYS):
    atms_today = atms_needing_service[day]
    random.shuffle(atms_today)
    group_routes = {group_id: [] for group_id in range(NUM_GROUPS)}
    group_times = {group_id: 0 for group_id in range(NUM_GROUPS)}
    unassigned_atms = []

'''Для каждого дня мы распределяем банкоматы между 5 группами инкассаторов, следя за тем, чтобы продолжительность работы каждой группы не превышала 8 часов.
atms_today: Банкоматы, которые требуют обслуживания в текущий день.
group_routes и group_times: Хранение маршрута и времени для каждой группы.
unassigned_atms: Банкоматы, которые не удалось назначить на маршрут из-за ограничения по времени.'''

'Для каждого дня мы распределяем банкоматы между 5 группами инкассаторов, следя за тем, чтобы продолжительность работы каждой группы не превышала 8 часов.\natms_today: Банкоматы, которые требуют обслуживания в текущий день.\ngroup_routes и group_times: Хранение маршрута и времени для каждой группы.\nunassigned_atms: Банкоматы, которые не удалось назначить на маршрут из-за ограничения по времени.'

In [22]:
for atm in atms_today:
    assigned = False
    for group_id in range(NUM_GROUPS):
        if group_routes[group_id]:
            last_atm = group_routes[group_id][-1]
            current_pos = (last_atm.x, last_atm.y)
        else:
            current_pos = (0, 0)  # База

        distance = ((atm.x - current_pos[0]) ** 2 + (atm.y - current_pos[1]) ** 2) ** 0.5
        travel_time = (distance / AVERAGE_SPEED) * 60
        service_time = SERVICE_TIME_PER_ATM
        potential_total_time = group_times[group_id] + travel_time + service_time

        if potential_total_time <= WORKING_DAY_DURATION:
            group_routes[group_id].append(atm)
            group_times[group_id] = potential_total_time
            assigned = True
            break
    if not assigned:
        unassigned_atms.append(atm)


'''Распределение банкоматов: Каждый банкомат проходит проверку на назначение одной из групп.
Рассчет времени на перемещение: Если в маршруте группы уже есть банкомат, то время рассчитывается как расстояние между текущим банкоматом и последним в маршруте (используя формулу Евклидова расстояния).
Это расстояние делится на среднюю скорость и умножается на 60, чтобы получить время в минутах.
Добавление в маршрут: Если общее время группы (включая обслуживание и перемещение) не превышает рабочего дня, банкомат добавляется в маршрут этой группы, и общее время маршрута обновляется.'''

'Распределение банкоматов: Каждый банкомат проходит проверку на назначение одной из групп.\nРассчет времени на перемещение: Если в маршруте группы уже есть банкомат, то время рассчитывается как расстояние между текущим банкоматом и последним в маршруте (используя формулу Евклидова расстояния).\nЭто расстояние делится на среднюю скорость и умножается на 60, чтобы получить время в минутах.\nДобавление в маршрут: Если общее время группы (включая обслуживание и перемещение) не превышает рабочего дня, банкомат добавляется в маршрут этой группы, и общее время маршрута обновляется.'

# **Вывод**

In [23]:
for day in range(NUM_DAYS):
    print(f"Day {day + 1} Routes:")
    for group_id, route in group_routes.items():
        print(f"  Group {group_id + 1} Route: {[atm.atm_id for atm in route]}")
    if unassigned_atms:
        print("  Unassigned ATMs:", [atm.atm_id for atm in unassigned_atms])
    print("\n")

Day 1 Routes:
  Group 1 Route: [513, 979, 820, 83, 305, 66, 734, 91, 258, 666, 376, 890, 357, 35, 409, 176, 269, 826]
  Group 2 Route: [576, 696, 455, 817, 290, 421, 928, 191, 442, 886, 352, 377, 612, 482, 672, 374, 935, 618, 743]
  Group 3 Route: [952, 883, 234, 994, 912, 96, 193, 771, 47, 568, 46, 889, 885, 926, 97, 158, 816, 444, 74]
  Group 4 Route: [307, 326, 65, 737, 110, 23, 718, 198, 152, 351, 868, 519, 894, 963, 336, 897, 667, 498]
  Group 5 Route: [953, 338, 26, 378, 707, 367, 318, 574, 302, 787, 483, 161, 294, 52, 364, 157, 456]
  Unassigned ATMs: [493, 968, 391, 128, 956, 878, 744, 762, 196, 831, 286, 524, 189, 862, 544, 809, 964, 595, 892, 187, 636, 384, 3, 864, 756, 879, 36, 514, 382, 925, 587, 623, 19, 564, 966, 505, 634, 796, 412, 936, 310, 153, 775, 682, 186, 751, 64, 929, 870, 194, 882, 144, 961, 216, 728, 752, 893, 165, 729, 479, 649, 486, 735, 578, 725, 254, 557, 453, 296, 689, 845, 370, 599, 547, 105, 1, 334, 829, 923, 170, 106, 100, 188, 757, 55, 976, 774, 520, 16

Дополнение с алгоритмом Насти

In [3]:
import numpy as np
import heapq

# Параметры
NUM_ATMS = 100  # Количество банкоматов
NUM_GROUPS = 5  # Количество групп инкассаторов
NUM_DAYS = 3  # Количество дней, на которые нужно построить маршруты
WORKING_DAY_DURATION = 8 * 60  # Рабочий день в минутах
SERVICE_TIME_PER_ATM = 10  # Время обслуживания одного банкомата, например, 10 минут

class ATM:
    def __init__(self, atm_id, accept_capacity, dispense_capacity, accept_rate, dispense_rate):
        self.atm_id = atm_id
        self.accept_capacity = accept_capacity
        self.dispense_capacity = dispense_capacity
        self.accept_rate = accept_rate
        self.dispense_rate = dispense_rate
        self.current_accept = np.random.randint(0, accept_capacity)  # Начальное количество для приема
        self.current_dispense = np.random.randint(0, dispense_capacity)  # Начальное количество для выдачи

    def needs_service(self):
        # Условие на необходимость обслуживания
        return self.current_accept >= self.accept_capacity or self.current_dispense <= 0

    def update(self):
        # Моделирование изменения состояния банкомата за день
        self.current_accept += int(np.random.normal(self.accept_rate))
        self.current_dispense -= int(np.random.normal(self.dispense_rate))

# Инициализация банкоматов
np.random.seed(1)
atms = [ATM(i, np.random.randint(100, 500), np.random.randint(100, 500),
             np.random.randint(10, 30), np.random.randint(10, 30)) for i in range(NUM_ATMS)]

# Создание дорожной сети
adj_matrix = np.random.randint(1, 101, size=(NUM_ATMS + 1, NUM_ATMS + 1))  # включая базу
np.fill_diagonal(adj_matrix, 0)  # расстояние от узла до самого себя - 0

# Функция Дейкстры для нахождения кратчайшего пути между банкоматами
def dijkstra(matrix, start_id, end_id):
    n = len(matrix)
    distances = {i: float('infinity') for i in range(n)}
    distances[start_id] = 0
    priority_queue = [(0, start_id)]
    predecessors = {i: None for i in range(n)}

    while priority_queue:
        current_distance, current_node = heapq.heappop(priority_queue)
        if current_distance > distances[current_node]:
            continue

        for neighbor, weight in enumerate(matrix[current_node]):
            if weight > 0:
                distance = current_distance + weight
                if distance < distances[neighbor]:
                    distances[neighbor] = distance
                    predecessors[neighbor] = current_node
                    heapq.heappush(priority_queue, (distance, neighbor))

    # Восстанавливаем путь
    path = []
    current = end_id
    while current is not None:
        path.append(current)
        current = predecessors[current]

    return path[::-1]

# Функция для планирования маршрутов инкассаторов на каждый день
def plan_routes(atms, adj_matrix, num_groups, num_days):
    base_location = NUM_ATMS  # База инкассаторов
    for day in range(num_days):
        print(f"Day {day + 1} Routes:")

        # Определяем банкоматы, которые нуждаются в обслуживании
        atms_needing_service = [atm.atm_id for atm in atms if atm.needs_service()]

        # Инициализируем маршруты и время для каждой группы инкассаторов
        group_routes = {group_id: [] for group_id in range(num_groups)}
        group_times = {group_id: 0 for group_id in range(num_groups)}

        while atms_needing_service:
            for group_id in range(num_groups):
                if not atms_needing_service:
                    break  # Если больше нет банкоматов для обслуживания, заканчиваем цикл

                # Текущее местоположение инкассаторов (последний обслуженный банкомат или база)
                current_location = group_routes[group_id][-1] if group_routes[group_id] else base_location

                # Находим ближайший банкомат из оставшихся нуждающихся в обслуживании
                shortest_path = None
                shortest_time = float('inf')
                closest_atm = None

                for atm_id in atms_needing_service:
                    path = dijkstra(adj_matrix, current_location, atm_id)
                    time = sum(adj_matrix[path[i]][path[i + 1]] for i in range(len(path) - 1))

                    if time < shortest_time:
                        shortest_time = time
                        shortest_path = path
                        closest_atm = atm_id

                if closest_atm is None:
                    continue  # Если банкоматы закончились, пропускаем

                # Проверяем, помещается ли время на обслуживание в рабочий день
                potential_total_time = group_times[group_id] + shortest_time + SERVICE_TIME_PER_ATM

                if potential_total_time <= WORKING_DAY_DURATION:
                    group_routes[group_id].extend(shortest_path[1:])  # Добавляем маршрут к группе
                    group_times[group_id] = potential_total_time
                    atms_needing_service.remove(closest_atm)  # Убираем банкомат из списка нуждающихся
                else:
                    # Если банкомат не помещается по времени, завершаем маршрут для этой группы
                    break

        # Вывод маршрутов группы на текущий день
        for group_id, route in group_routes.items():
            print(f"  Group {group_id + 1} Route: {route}")

        # Обновляем состояние банкоматов в конце дня
        for atm in atms:
            atm.update()

# Планируем маршруты на несколько дней
plan_routes(atms, adj_matrix, NUM_GROUPS, NUM_DAYS)

Day 1 Routes:
  Group 1 Route: []
  Group 2 Route: []
  Group 3 Route: []
  Group 4 Route: []
  Group 5 Route: []
Day 2 Routes:
  Group 1 Route: [48, 70, 61, 49, 57, 78, 80]
  Group 2 Route: [74, 14, 71, 79, 0, 71, 63, 6, 7]
  Group 3 Route: [73, 18, 19, 31, 62]
  Group 4 Route: [73, 33, 35, 55, 13, 30, 64]
  Group 5 Route: [74, 14, 71, 63, 65, 77, 94]
Day 3 Routes:
  Group 1 Route: [48, 70, 61, 49, 84, 83, 19, 31, 62]
  Group 2 Route: [9, 36, 68, 49, 30, 41, 1, 24, 8, 91, 34, 17, 65]
  Group 3 Route: [74, 14, 71, 63, 6, 7, 80, 83, 19, 15, 77]
  Group 4 Route: [73, 18, 90, 69, 37, 33, 35, 55, 13, 59, 90, 91, 34, 16, 13, 59, 94]
  Group 5 Route: [73, 35, 86, 64, 22, 76, 75, 0]
